## MATH 462 - Homework 4

Augusto Gonzalez-Bonorino

### Part I

Chapter 6, exercises 7 and 8.

#### Exercise 6

To streamline the experiments varying data set size, i wrote a utility function that parameterizes this argument.

*run_experiment* takes in an integer, denoting the size of the dataset to generate, and then uses scikit-learn's *make_moons* method to create a clustered dataset from classification. After proper data splitting, grid search cross validation is used to find the best hyperparameters for a decision tree classifier. The function returns the best tree model estimated, it's accuracy score, and the data splits for convenience.

In [ ]:
# 6 task: change size of data set

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

## Experiment varying data set size
def run_experiment(data_size):

    X_moons, y_moons = make_moons(n_samples=data_size,
                              noise=0.5,
                              random_state=42)

    X_train, X_test, y_train, y_test = train_test_split(X_moons, y_moons,
                                                    test_size=0.2,
                                                    random_state=42)

    params = {
        'max_leaf_nodes': list(range(2, 10)),
        'max_depth': list(range(1, 4)),
        'min_samples_split': [2, 3]
    }
    grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42),
                                params,
                                cv=3)

    grid_search_cv.fit(X_train, y_train)

    best_model = grid_search_cv.best_estimator_
    accuracy = accuracy_score(grid_search_cv.predict(X_test), y_test)

    split = (X_train, X_test, y_train, y_test)

    return best_model, accuracy, split

sizes = [1_000, 10_000, 100_000, 1_000_000]
data_splits = []
tree_models = []

for size in sizes:
    best_model, accuracy, split = run_experiment(size)
    data_splits.append(split)
    tree_models.append(best_model)

    print(f"\nBest performing decision tree ({size}): \n\n- {best_model}")
    print(f"Performance metrics: \n\n- Accuracy: {accuracy}")


Best performing decision tree (1000): 

- DecisionTreeClassifier(max_depth=2, max_leaf_nodes=4, random_state=42)
Performance metrics: 

- Accuracy: 0.79

Best performing decision tree (10000): 

- DecisionTreeClassifier(max_depth=2, max_leaf_nodes=4, random_state=42)
Performance metrics: 

- Accuracy: 0.823

Best performing decision tree (100000): 

- DecisionTreeClassifier(max_depth=2, max_leaf_nodes=4, random_state=42)
Performance metrics: 

- Accuracy: 0.81745

Best performing decision tree (1000000): 

- DecisionTreeClassifier(max_depth=2, max_leaf_nodes=4, random_state=42)
Performance metrics: 

- Accuracy: 0.818965


Interestingly, the accuracy of the decision tree classifier slightly drops with large datasets. The best model performed best for a dataset of size 10_000. I could consider retraining these models with a wider range of available values for the hyperparameters, and assess if efficacy changes with cross validation.

#### Exercise 8

Following a similar coding practice as in the previous exercise, I create a series of utility functions to streamline the experiments. The function *build_mini_sets* takes in the number of instances to include in each split, the variable with the splits from the previous exercise, and the number of trees for populating the forest. It then creates the randomly shuffled "mini" training sets by choosing random indices to subset the original training matrix. Finally, a list with the mini training sets is returned.

A second function, *build_forest*, is used te the forest via the method employed in the book. We create a list of clones of the best model we estimated in the previous exercise, train each cloned tree on the mini training sets and compute accuracy score for each. The function returns the forest model and the average accuracy score of all trees trained.

Lastly, the *majority_vote_pred* function takes in the forest just created, the number of trees in the forest, and the original data splits. It creates a new list with the individual predictions or "votes" of each tree. A majority vote technique is implemented by computing the mode classication and comparing them to the true test values to estimate an overall accuracy score of the random forest via majority voting.

In [ ]:
# 8 task: change size of n_instances randomly selected from training set

from sklearn.model_selection import ShuffleSplit
from sklearn.base import clone
import numpy as np
from scipy.stats import mode

def build_mini_sets(size, data_split, num_trees):

    n_instances = size

    mini_sets = []

    rs = ShuffleSplit(n_splits=num_trees, test_size=len(data_split[0]) - n_instances,
                    random_state=42)

    for mini_train_index, mini_test_index in rs.split(data_split[0]):
        X_mini_train = data_split[0][mini_train_index]
        y_mini_train = data_split[2][mini_train_index]
        mini_sets.append((X_mini_train, y_mini_train))

    return mini_sets

def build_forest(tree_model, num_trees, mini_sets, data_split):

    forest = [clone(tree_model) for _ in range(num_trees)]

    accuracy_scores = []

    for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
        tree.fit(X_mini_train, y_mini_train)

        y_pred = tree.predict(data_split[1])
        accuracy_scores.append(accuracy_score(data_split[3], y_pred))

    avg_acc = np.mean(accuracy_scores)

    return forest, avg_acc

def majority_vote_pred(forest, num_trees, data_split):

    Y_pred = np.empty([num_trees, len(data_split[1])], dtype=np.uint8)

    for tree_index, tree in enumerate(forest):
        Y_pred[tree_index] = tree.predict(data_split[1])

    y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)
    score = accuracy_score(data_split[3], y_pred_majority_votes.reshape([-1]))

    return score

In [ ]:
k = 10
for dataset, tree_model in zip(data_splits, tree_models):
    size = 10 * k
    n_trees = 1000

    mini_sets = build_mini_sets(size, dataset, n_trees)

    forest, avg_acc = build_forest(tree_model, n_trees, mini_sets, dataset)
    print(f"Average accuracy ({size}): {avg_acc}\n")

    majority_vote = majority_vote_pred(forest, n_trees, dataset)
    print(f"Majority vote accuracy ({size}): {majority_vote}\n")

    k = k*10

Average accuracy (100): 0.7661950000000001

Majority vote accuracy (100): 0.805

Average accuracy (1000): 0.821665

Majority vote accuracy (1000): 0.8245

Average accuracy (10000): 0.81727035

Majority vote accuracy (10000): 0.82015

Average accuracy (100000): 0.818873075

Majority vote accuracy (100000): 0.819



We can see minor improvements from majority vote, compared to a simple average prediction, on accuracy. Still, the interesting phenomena of decreased performance on bigger datasets hold. Nevertheless, the difference throughout different dataset sizes is smaller (changes observed at the three decimal place) so majority voting provides slightly better but more consistent scores.

### Part II

New dataset:

In [ ]:
%pip install kaggle

In [ ]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download nimapourmoradi/steel-dataset

  0% 0.00/484k [00:00<?, ?B/s]
100% 484k/484k [00:00<00:00, 111MB/s]


## Steel dataset

The information gathered is from the DAEWOO Steel Co. Ltd in Gwangyang, South Korea. It produces several types of coils, steel plates, and iron plates. The information on electricity consumption is held in a cloud-based system. The information on energy consumption of the industry is stored on the website of the Korea Electric Power Corporation (pccs.kepco.go.kr), and the perspectives on daily, monthly, and annual data are calculated and shown.

The goal is to classify load type on one of the following class labels: Light Load, Medium Load, Maximum Load.

This is an interesting dataset because it contains variables of varying type (float, integer, and strings), thus it requires preprocessing for standardizing numerical features (Usage_kWh, Lagging_Current_Reactive.Power_kVarh, Leading_Current_Reactive_Power_kVarh, CO2 (tCO2), Lagging_Current_Power_Factor, Leading_Current_Power_Factor, NSM) and encoding text labels (WeekStatus, Day_Of_Week).

It has 35041 observations and no missing values. To follow the book's exercise, I will split the dataset into a similar ratio of 25:5:5 for building the training, validation, and testing sets.

----

The data can be obtained from Kaggle -> https://www.kaggle.com/datasets/nimapourmoradi/steel-dataset

Efficient energy consumption prediction model for a data analytic-enabled industry building in a smart city By Sathishkumar V E, Changsun Shin, Yongyun Cho. 2021

Published in Building Research & Information, Vol. 49. no. 1, pp. 127-143

In [ ]:
import zipfile

def unzip_and_extract(file_path):
  """
  Unzips a folder and extracts the information.

  Args:
      file_path: The path to the zip file.
  """

  # Open the zip file
  with zipfile.ZipFile(file_path, 'r') as zip_ref:
    # Extract all files
    zip_ref.extractall()

unzip_and_extract("/content/steel-dataset.zip")

In [ ]:
# 8 Voting Classifier on Steel Dataset

import pandas as pd

data = pd.read_csv('/content/Steel_industry.csv')
data.head()

,Date_Time,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_Of_Week,Load_Type
0,01/01/2018 00:15,3.17,2.95,0.0,0.0,73.21,100.0,900,Weekday,Monday,Light_Load
1,01/01/2018 00:30,4.00,4.46,0.0,0.0,66.77,100.0,1800,Weekday,Monday,Light_Load
2,01/01/2018 00:45,3.24,3.28,0.0,0.0,70.28,100.0,2700,Weekday,Monday,Light_Load
3,01/01/2018 01:00,3.31,3.56,0.0,0.0,68.09,100.0,3600,Weekday,Monday,Light_Load
4,01/01/2018 01:15,3.82,4.50,0.0,0.0,64.72,100.0,4500,Weekday,Monday,Light_Load


In [ ]:
data[['WeekStatus', 'Day_Of_Week']].nunique()

WeekStatus     2
Day_Of_Week    7
dtype: int64

In [ ]:
X, y = data.iloc[:, 1:10].to_numpy(), data.iloc[:, 10:].to_numpy()

data.dtypes, X.shape, y.shape

(Date_Time                                object
 Usage_kWh                               float64
 Lagging_Current_Reactive.Power_kVarh    float64
 Leading_Current_Reactive_Power_kVarh    float64
 CO2(tCO2)                               float64
 Lagging_Current_Power_Factor            float64
 Leading_Current_Power_Factor            float64
 NSM                                       int64
 WeekStatus                               object
 Day_Of_Week                              object
 Load_Type                                object
 dtype: object,
 (35041, 9),
 (35041, 1))

In [ ]:
# Split into train, val, test
## 50:10:10 ratio for MNIST; 25:5:5 ratio for Steel
X_train, y_train = X[:25_000], y[:25_000]
X_val, y_val = X[25_000:30_000], y[25_000:30_000]
X_test, y_test = X[30_000:], y[30_000:]

In [ ]:
X_train[0] # one unprocessed training observation

array([3.17, 2.95, 0.0, 0.0, 73.21, 100.0, 900, 'Weekday', 'Monday'],
      dtype=object)

### Preprocessing

#### Numeric Features Preprocessing

The numeric_features_idx indicates the indexes of numeric columns in the dataset, which are subjected to two main preprocessing steps within a Pipeline:

**Imputation**: Uses SimpleImputer with a strategy of 'median' to fill in any missing values with the median of the column. Even though it's mentioned that the dataset has no missing values, this step could be a precautionary measure to handle potential anomalies or future data that might have missing entries.

**Scaling**: Applies StandardScaler to normalize the numeric features. This scaler removes the mean and scales the features to unit variance. This step is crucial for models that are sensitive to the magnitude of variables, ensuring that all numeric features contribute equally to the model's performance.

#### Categorical Features Preprocessing

The *categorical_features_idx* lists the indexes of categorical columns, which go through their preprocessing pipeline:

**Imputation**: Similarly, ***SimpleImputer*** is used but with a strategy of 'constant' and a fill_value of 'missing'. This approach handles any potential missing values by assigning them a constant label ('missing'), ensuring that the encoder can process these values later.

**One-Hot Encoding**: Utilizes ***OneHotEncoder*** to convert categorical variables into a format that can be provided to ML algorithms. Since machine learning models require numerical input, this step transforms categorical variables into a binary matrix representing the presence (or absence) of a category. This is particularly important for models that cannot handle categorical values directly.

#### Column Transformation

***ColumnTransformer*** is then used to apply these preprocessing steps to their respective columns in the dataset. This transformer allows for different columns or column subsets of the input to be transformed separately and the results concatenated into a single feature space. This is especially useful in datasets like this one, where we have a mix of numerical and categorical inputs.


The transformed features are applied to the training, validation, and testing sets. Notably, the validation and testing sets use the .transform() method instead of .fit_transform() for the testing set to ensure that they are scaled or encoded based on the parameters learned from the training set, preventing data leakage.

#### Label Encoding

Finally, the target labels (y_train, y_val, y_test) are encoded using ***LabelEncoder***. This encoder converts the class labels into integers, which is necessary for most machine learning models in sklearn that require numerical labels. It's important to note that the encoder is fitted only on the training data to establish a consistent mapping of classes to integers, then applied to both training and testing labels to ensure consistency.

In [ ]:
# data processing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import numpy as np

numeric_features_idx = list(range(0, 7))
categorical_features_idx = list(range(7, 9))

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# Define the preprocessing for categorical features (encode them)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder())])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features_idx),
        ('cat', categorical_transformer, categorical_features_idx)])


X_train_transformed = preprocessor.fit_transform(X_train)
X_val_transformed = preprocessor.fit_transform(X_val)
X_test_transformed = preprocessor.transform(X_test)

label_encoder = LabelEncoder()

# Fit the encoder on the training data and transform both training and test labels
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.fit_transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [ ]:
X_train_transformed[0], y_train_encoded[0] # 7 numerical + 9 one hot encoded (7 days and 2 week status)

(array([-0.73996798, -0.62276433, -0.51506717, -0.73103916, -0.39928842,
         0.50404433, -1.67629364,  1.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ]),
 0)

I consider three models to build the ensemble: Logistic Regression, Support Vector Classifier, and a Decision Tree Classifier.

The choice was motivated by the multinomial classification task, we have multiple class labels any given observation can be classified into, and because of their mainstream usage in the literature for this task.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

estimators = []
estimators.append(('LR',
                  LogisticRegression(solver ='lbfgs',
                                     multi_class ='multinomial',
                                     max_iter = 200)))
estimators.append(('SVC', SVC(gamma ='auto',
                              probability = True)))
estimators.append(('DTC', DecisionTreeClassifier()))

for estimator, fitm in estimators:
    print("Training the", estimator)
    fitm.fit(X_train_transformed, y_train_encoded)


Training the LR
Training the SVC
Training the DTC


In [ ]:
# note the logistic regression performs significantly worse than SVC and DTC.
# thus I will only consider the latter two for building my ensemble

[fitm.score(X_val_transformed, y_val_encoded) for estimator, fitm in estimators]

[0.7768, 0.8312, 0.851]

In [ ]:
from sklearn.ensemble import VotingClassifier

# Voting Classifier with soft voting
vot_hard = VotingClassifier(estimators = estimators[1:], voting ='hard')
vot_hard.fit(X_train_transformed, y_train_encoded)

# Voting Classifier with soft voting
vot_soft = VotingClassifier(estimators = estimators[1:], voting ='soft')
vot_soft.fit(X_train_transformed, y_train_encoded)


VotingClassifier(estimators=[('SVC', SVC(gamma='auto', probability=True)),
                             ('DTC', DecisionTreeClassifier())],
                 voting='soft')

In [ ]:
vot_hard.score(X_val_transformed, y_val_encoded)

0.8354

In [ ]:
vot_soft.score(X_val_transformed, y_val_encoded)

0.8494

Voting classifiers are a type of ensemble learning technique where multiple models, often of different types, are trained on the same data and then used to predict the output class. The final output class is determined based on the votes from all the models. The main idea is to combine the decision-making capabilities of various models to reduce overfitting, increase robustness, and improve the overall prediction accuracy.

There are two main types of voting:

**Hard Voting**: In hard voting, each model in the ensemble votes for a single class, and the class with the majority of the votes is chosen as the final prediction. This method does not take into account the confidence of the votes; it only counts the number of votes.

**Soft Voting**: Soft voting, on the other hand, takes into account the probability estimates (confidence) of the vote. Each model's vote is weighted by its confidence in the prediction. The final output class is the one with the highest sum of predicted probabilities. This method is often more flexible and achieves higher performance because it leverages the predictive confidence of each model.

#### Code Explanation

The code initializes three different classifiers: Logistic Regression, Support Vector Classifier (SVC), and Decision Tree Classifier (DTC), each with specific hyperparameters suited to the dataset and the task at hand. These classifiers are added to a list called estimators.

- **Logistic Regression (LR)**: Configured for multinomial classification, suitable for multiclass problems.
- **Support Vector Classifier (SVC)**: Enabled with probability=True to allow the use of soft voting, as it requires probability estimates for each class.
- **Decision Tree Classifier (DTC)**: A straightforward implementation without specified hyperparameters, demonstrating its versatility.

Each classifier is then trained (fit) on the transformed training dataset (X_train_transformed) and the encoded target labels (y_train_encoded).

##### Ensemble Models with Voting Classifier

Two instances of the ***VotingClassifier*** are created, differing in their voting strategies: one uses hard voting (vot_hard), and the other employs soft voting (vot_soft). Both are configured with a subset of the previously defined classifiers.

After instantiation, both voting classifiers are trained on the same dataset. This training process leverages the strengths of the included classifiers to make more accurate predictions based on the collective decision-making process defined by the voting strategy.

The performance of the hard voting classifier is evaluated on a validation set (X_val_transformed, y_val_encoded), demonstrating its ability to generalize the learned patterns to new data. It achieves a score of 0.8346. Similarly, the soft voting classifier is evaluated, achieving a higher score of 0.8586, indicating that, in this case, leveraging the confidence levels of the predictions (soft voting) provides a more accurate model than merely counting votes (hard voting).

In [ ]:
# 9 Stacking Ensemble on Steel Dataset

X_valid_predictions = np.empty((len(X_val_transformed), len(estimators)), dtype=object)
idx = 0
for estimator, fitm in estimators:
    print(idx)
    X_valid_predictions[:, idx] = fitm.predict(X_val_transformed)
    idx += 1

0
1
2


In [ ]:
X_valid_predictions

array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       ...,
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]], dtype=object)

In [ ]:
rnd_forest_blender = RandomForestClassifier(n_estimators=200, oob_score=True,
                                            random_state=42)
rnd_forest_blender.fit(X_valid_predictions, y_val_encoded)

RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)

In [ ]:
rnd_forest_blender.oob_score_

0.872

In [ ]:
X_test_predictions = np.empty((len(X_test_transformed), len(estimators)), dtype=object)
idx = 0
for estimator, fitm in estimators:
    X_test_predictions[:, idx] = fitm.predict(X_test_transformed)
    idx += 1

In [ ]:
y_pred = rnd_forest_blender.predict(X_test_predictions)
accuracy_score(y_test_encoded, y_pred)

0.7266415393771077

In [ ]:
## with stacking classifier
### Since StackingClassifier uses K-Fold cross-validation, we don't need a separate validation set,
###so let's join the training set and the validation set into a bigger training set:
from sklearn.ensemble import StackingClassifier

X_train_full, y_train_full = X[:30_000], y[:30_000]
X_train_full_transformed = preprocessor.fit_transform(X_train_full)
y_train_full_encoded = label_encoder.fit_transform(y_train_full)

stack_clf = StackingClassifier(estimators,
                               final_estimator=rnd_forest_blender)
stack_clf.fit(X_train_full_transformed, y_train_full_encoded)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


StackingClassifier(estimators=[('LR',
                                LogisticRegression(max_iter=200,
                                                   multi_class='multinomial')),
                               ('SVC', SVC(gamma='auto', probability=True)),
                               ('DTC', DecisionTreeClassifier())],
                   final_estimator=RandomForestClassifier(n_estimators=200,
                                                          oob_score=True,
                                                          random_state=42))

In [ ]:
stack_clf.score(X_test_transformed, y_test_encoded)

0.6881571116841897

#### Random Forest

A Random Forest is an ensemble learning method that operates by constructing a multitude of decision trees at training time and outputting the class that is the mode of the classes (classification) or mean prediction (regression) of the individual trees. Random forests correct for decision trees' habit of overfitting to their training set. They are highly versatile, can handle both classification and regression tasks, and can manage datasets with a mix of numerical and categorical features. The oob_score (Out-of-Bag score) is a measure of prediction accuracy that can be used as an alternative to cross-validation. It is calculated using predictions from the trees that did not use a given data point during training, providing an efficient estimate of the model's performance.

#### Stacking

Stacking involves training a new model to aggregate the predictions of several base models. The base models are trained on the full training set, then a new model (called the blender or meta-learner) is trained on the outputs of these base models as features. This approach can lead to better predictive performance compared to using any single model alone because it combines the strengths of various models. The key idea is that the blender learns how to best combine the predictions from the base models.

#### Comparison of Ensemble Models

In the literature, both random forests and stacking are well-regarded for their ability to improve prediction accuracy through ensemble methods. Studies and empirical evidence suggest that ensemble methods can outperform individual estimators by reducing variance (bagging, random forests), reducing bias (boosting), or increasing the predictive force through diversity (stacking). Each method has its context where it shines, with stacking often being highlighted for its ability to blend the predictive power of highly varied models through a meta-learner.

In the provided code, stacking is implemented manually using predictions from multiple estimators as input features for a random forest blender. The estimators (Logistic Regression, SVC, and Decision Tree Classifier) predict the validation set, and these predictions are used as features for the random forest model to fit. This process is called blending when using a hold-out set (like a validation set) to train the blender.

I found it quite interesting that the accuracy score dropped significantly in the stacking model. Some potential causes of this discrepancy include:

- **Overfitting in Blender Model**: The Random Forest blender might be overfitting the validation predictions (X_valid_predictions). An OOB score of 0.8718 suggests the blender performs well on the training data, but this might not generalize well to unseen data, as observed with the test set accuracy (0.7445).

- **Mismatch in Data Distribution**: The stacking classifier's performance might suffer due to differences in the distribution of the training set (including the validation set now) and the test set. Joining the training and validation sets into a bigger training set changes the data's distribution, which the final estimator is trained on.

- **Complexity and Diversity of Base Estimators**: The effectiveness of stacking and blending techniques heavily depends on the diversity and accuracy of the base estimators. If the base estimators are too correlated or if there's not enough diversity in their predictions, the final model may not significantly improve or could even perform worse.